In [0]:
%sql
USE CATALOG newpavancatalog;
use schema bronze;

In [0]:
%sql
USE CATALOG newpavancatalog;
USE SCHEMA bronze;

CREATE EXTERNAL VOLUME IF NOT EXISTS volume_auto_loader_payments
    LOCATION 'abfss://landing@parkaru15sa.dfs.core.windows.net/payments';

In [0]:
%fs ls /Volumes/newpavancatalog/bronze/volume_auto_loader_payments/

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

schema = StructType([
    StructField("payment_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("payment_timestamp", TimestampType(), True),
    StructField("payment_status", IntegerType(), True),
    StructField("payment_method", StringType(), True)
])

In [0]:
customers_df = (
                    spark.readStream
                         .format("cloudFiles")
                         .option("cloudFiles.format", "csv")
                         .option("cloudFiles.schemaLocation", "/Volumes/newpavancatalog/bronze/volume_auto_loader_payments/_schema")
                         .option("cloudFiles.inferColumnTypes", "true")
                         .option("cloudFiles.schemaHints", "payment_timestamp TIMESTAMP")
                         .load("/Volumes/newpavancatalog/bronze/volume_auto_loader_payments/")
)

In [0]:
display(customers_df)

In [0]:
from pyspark.sql.functions import current_timestamp, col

customers_transformed_df = (
                                customers_df.withColumn("file_path", col("_metadata.file_path"))
                                            .withColumn("ingestion_date", current_timestamp())
)